In [0]:
import tensorflow as tf

In [5]:
tf.__version__

'2.0.0'

In [5]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 86.3MB 55.6MB/s 
     |████████████████████████████████| 3.8MB 27.1MB/s 
     |████████████████████████████████| 450kB 43.1MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=5ab555aed92ae6f75dfd0155489797fd3dab16faa713b63ec4fa98a5e4a7fe8e
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.2
    Uninstalling gast-0.3.2:
      Successfully uninstalled gast-0.3.2
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
!ls drive/My\ Drive/Deep\ learning\ dataset/

'Google Landmark Recognition'   nyu-cv-fall-2018.zip   trim_data


In [5]:
!unzip drive/My\ Drive/Deep\ learning\ dataset/nyu-cv-fall-2018.zip

Archive:  drive/My Drive/Deep learning dataset/nyu-cv-fall-2018.zip
  inflating: test_images.zip         
  inflating: train_images.zip        


In [6]:
!unzip /content/train_images.zip

Archive:  /content/train_images.zip
warning [/content/train_images.zip]:  150994944 extra bytes at beginning or within zipfile
  (attempting to process anyway)
   creating: train_images/
   creating: train_images/00036/
  inflating: train_images/00036/00004_00013.ppm  
  inflating: train_images/00036/00007_00027.ppm  
  inflating: train_images/00036/00002_00000.ppm  
  inflating: train_images/00036/00000_00006.ppm  
  inflating: train_images/00036/00003_00024.ppm  
  inflating: train_images/00036/00006_00005.ppm  
  inflating: train_images/00036/00005_00019.ppm  
  inflating: train_images/00036/00001_00022.ppm  
  inflating: train_images/00036/00004_00024.ppm  
  inflating: train_images/00036/00005_00013.ppm  
  inflating: train_images/00036/00009_00005.ppm  
  inflating: train_images/00036/00004_00028.ppm  
  inflating: train_images/00036/00005_00011.ppm  
  inflating: train_images/00036/00005_00000.ppm  
  inflating: train_images/00036/00001_00029.ppm  
  inflating: train_images/0003

In [5]:
!ls

drive	     test_images      train_images
sample_data  test_images.zip  train_images.zip


In [8]:
!unzip /content/test_images.zip

Archive:  /content/test_images.zip
   creating: test_images/
  inflating: test_images/02879.ppm   
  inflating: test_images/01855.ppm   
  inflating: test_images/04406.ppm   
  inflating: test_images/07029.ppm   
  inflating: test_images/10142.ppm   
  inflating: test_images/01773.ppm   
  inflating: test_images/05587.ppm   
  inflating: test_images/01839.ppm   
  inflating: test_images/10556.ppm   
  inflating: test_images/12554.ppm   
  inflating: test_images/11827.ppm   
  inflating: test_images/11410.ppm   
  inflating: test_images/08328.ppm   
  inflating: test_images/12478.ppm   
  inflating: test_images/05949.ppm   
  inflating: test_images/09737.ppm   
  inflating: test_images/11102.ppm   
  inflating: test_images/07540.ppm   
  inflating: test_images/06507.ppm   
  inflating: test_images/09154.ppm   
  inflating: test_images/04164.ppm   
  inflating: test_images/05292.ppm   
  inflating: test_images/07500.ppm   
  inflating: test_images/00185.ppm   
  inflating: test_images/01

In [6]:
from PIL import Image
import numpy as np
import os

train_images = []
train_labels = []

count = 0
for dirc in sorted(os.listdir('train_images')):
    if(count < 1):
        count += 1
        continue
    # counter = 0
    for img_path in os.listdir('train_images/'+str(dirc)):
        im = Image.open('train_images/'+str(dirc)+'/'+str(img_path))
        im = im.resize((100,100))
        train_images.append(np.array(im))
        train_labels.append(count-1)
        # counter += 1
        # if(counter == 50):
        #     break
    count += 1
    print('Reading '+str(count-2)+' dir')

Reading 0 dir
Reading 1 dir
Reading 2 dir
Reading 3 dir
Reading 4 dir
Reading 5 dir
Reading 6 dir
Reading 7 dir
Reading 8 dir
Reading 9 dir
Reading 10 dir
Reading 11 dir
Reading 12 dir
Reading 13 dir
Reading 14 dir
Reading 15 dir
Reading 16 dir
Reading 17 dir
Reading 18 dir
Reading 19 dir
Reading 20 dir
Reading 21 dir
Reading 22 dir
Reading 23 dir
Reading 24 dir
Reading 25 dir
Reading 26 dir
Reading 27 dir
Reading 28 dir
Reading 29 dir
Reading 30 dir
Reading 31 dir
Reading 32 dir
Reading 33 dir
Reading 34 dir
Reading 35 dir
Reading 36 dir
Reading 37 dir
Reading 38 dir
Reading 39 dir
Reading 40 dir
Reading 41 dir
Reading 42 dir


In [0]:
train_images = np.array(train_images)
train_labels = np.array(train_labels)

In [8]:
train_images.shape

(39209, 100, 100, 3)

In [9]:
train_labels.shape

(39209,)

In [0]:
train_labels = np.reshape(train_labels, (-1,1))

In [11]:
train_labels.shape

(39209, 1)

In [12]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder()
train_labels = enc.fit_transform(train_labels).toarray()

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [13]:
train_labels.shape

(39209, 43)

In [0]:
train_images = train_images / 255.0

In [0]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras import Model


train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)

In [0]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, padding='same', activation='relu')
        self.pool1 = MaxPool2D((2,2))
        self.conv2 = Conv2D(64, 3, padding='same', activation='relu')
        self.pool2 = MaxPool2D((2,2))
        self.flatten = Flatten()
        self.d1 = Dense(512, activation='relu')
        self.dropout1 = Dropout(0.4)
        self.d2 = Dense(128, activation='relu')
        self.dropout2 = Dropout(0.4)
        self.d3 = Dense(43, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.d1(x)
        x = self.dropout1(x)
        x = self.d2(x)
        x = self.dropout2(x)
        x = self.d3(x)
        return x

model = MyModel()

In [0]:
loss_object = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

In [0]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')

In [0]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [33]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
#         print(images.shape,labels.shape)  
        train_step(images, labels)
    
    model.save_weights('/content', save_format='tf')

    print('Epoch:',str(epoch+1),' Loss:',str(train_loss.result()),' Accuracy:',str(train_accuracy.result()*100))
    train_loss.reset_states()
    train_accuracy.reset_states()

Epoch: 1  Loss: tf.Tensor(0.30459243, shape=(), dtype=float32)  Accuracy: tf.Tensor(91.17904, shape=(), dtype=float32)
Epoch: 2  Loss: tf.Tensor(0.043029416, shape=(), dtype=float32)  Accuracy: tf.Tensor(98.70948, shape=(), dtype=float32)
Epoch: 3  Loss: tf.Tensor(0.036688898, shape=(), dtype=float32)  Accuracy: tf.Tensor(98.96452, shape=(), dtype=float32)
Epoch: 4  Loss: tf.Tensor(0.031063538, shape=(), dtype=float32)  Accuracy: tf.Tensor(99.14305, shape=(), dtype=float32)
Epoch: 5  Loss: tf.Tensor(0.027383167, shape=(), dtype=float32)  Accuracy: tf.Tensor(99.21957, shape=(), dtype=float32)


In [9]:
from PIL import Image
import numpy as np
import os

test_images = []

count = 0
for dirc in sorted(os.listdir('test_images')):
#     print(dirc)
    if(count < 1):
        count += 1
        continue
#     for img_path in os.listdir('train_images/'+str(dirc)):
    im = Image.open('test_images/'+str(dirc))
    im = im.resize((100,100))
    test_images.append(np.array(im))
#         train_labels.append(count-2)
#     count += 1
    print('Reading '+str(dirc))

Reading 00000.ppm
Reading 00001.ppm
Reading 00002.ppm
Reading 00003.ppm
Reading 00004.ppm
Reading 00005.ppm
Reading 00006.ppm
Reading 00007.ppm
Reading 00008.ppm
Reading 00009.ppm
Reading 00010.ppm
Reading 00011.ppm
Reading 00012.ppm
Reading 00013.ppm
Reading 00014.ppm
Reading 00015.ppm
Reading 00016.ppm
Reading 00017.ppm
Reading 00018.ppm
Reading 00019.ppm
Reading 00020.ppm
Reading 00021.ppm
Reading 00022.ppm
Reading 00023.ppm
Reading 00024.ppm
Reading 00025.ppm
Reading 00026.ppm
Reading 00027.ppm
Reading 00028.ppm
Reading 00029.ppm
Reading 00030.ppm
Reading 00031.ppm
Reading 00032.ppm
Reading 00033.ppm
Reading 00034.ppm
Reading 00035.ppm
Reading 00036.ppm
Reading 00037.ppm
Reading 00038.ppm
Reading 00039.ppm
Reading 00040.ppm
Reading 00041.ppm
Reading 00042.ppm
Reading 00043.ppm
Reading 00044.ppm
Reading 00045.ppm
Reading 00046.ppm
Reading 00047.ppm
Reading 00048.ppm
Reading 00049.ppm
Reading 00050.ppm
Reading 00051.ppm
Reading 00052.ppm
Reading 00053.ppm
Reading 00054.ppm
Reading 00

In [0]:
test_images = np.array(test_images)

In [11]:
test_images.shape

(12630, 100, 100, 3)

In [0]:
test_images = test_images / 255.0

In [19]:
model.load_weights('/content')

In [24]:
predictions = []
count = 0
for img in test_images:
  count += 1 
  print(count)
  img = img.reshape(1,100,100,3)
  predictions.append(np.argmax(model(img),axis=1))

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [34]:
!ls

assets	sample_data	test_images	 train_images	   variables
drive	saved_model.pb	test_images.zip  train_images.zip


In [0]:
predictions = np.array(predictions)

In [27]:
predictions.shape

(12630, 1)

In [0]:
import pandas as pd 

df = pd.DataFrame(predictions)

In [0]:
df.to_csv('/content/pred.csv')

In [32]:
!ls

assets	pred.csv     saved_model.pb  test_images.zip  train_images.zip
drive	sample_data  test_images     train_images     variables


In [33]:
df

,0
0,16
1,1
2,38
3,33
4,11
5,38
6,18
7,12
8,25
9,35
